# Gene Transcription Model

\begin{align*}
G &\xrightarrow[]{25} G + 25 \\
M &\xrightarrow[]{1000} M + P \\
P + P & \xrightarrow[]{0.001} D \\
M &\xrightarrow[]{0.1} \emptyset \\
P &\xrightarrow[]{1} \emptyset
\end{align*}

We gather the quantities into a vector
$$ X_t = \begin{pmatrix}
\text{# of $M$ molecues at time $t$} \\
\text{# of $P$ molecues at time $t$} \\
\text{# of $D$ molecues at time $t$}
\end{pmatrix} $$

In [77]:
import numpy as np

class Reaction(object):
    def __init__(self, reactants, products, ease):
        """
        reactants and products should be a list of integers
        """
        self.reactants = reactants
        self.products = products
        self.change = np.array(products) - np.array(reactants)
        self.ease = ease
        
    def rate(self, molecular_counts):
        """
        molecular_counts[n, k] should give the number of molecules of
        type k in the nth path. Returns rates such that rates[n] is the rate
        this reaction in the nth path.
        """
        N, _ = molecular_counts.shape
        rate = self.ease * np.ones(N)
        
        for reactant, amount in enumerate(self.reactants):
            for i in range(amount):
                rate = rate * (molecular_counts[:,reactant] - i)
        
        return rate
        
reactions = [([1, 0, 0, 0], [1, 1, 0, 0], 25),
             ([0, 1, 0, 0], [0, 1, 1, 0], 1000),
             ([0, 0, 2, 0], [0, 0, 0, 1], 0.001),
             ([0, 1, 0, 0], [0, 0, 0, 0], 0.1),
             ([0, 0, 1, 0], [0, 0, 0, 0], 1)]

reactions = [Reaction(*reaction) for reaction in reactions]

T = 1 # terminal time
n = 10**2  # number of time steps
N = 10**2 # number of paths
initial_counts = np.array([1, 0, 0, 0])

def tau_leaping(reactions, initial_counts, T, n, N):
    """
    Implements the tau leaping method for simulating stochatic
    biochemical equations
    """
    k = initial_counts.size # number of reactants
    h = T / n
    
    M = N # number of paths used this loop
    
    X = initial_counts * np.ones((M, k)) # molecular counts
    
    for _ in range(n):
        reactions_occured = []
        
        for reaction in reactions:
            reactions_occured.append(np.random.poisson(h * reaction.rate(X)))
            
        for reaction, r in zip(reactions, reactions_occured):
            X += np.outer(r, reaction.change)
            X = np.maximum(0, X) # due to approximation, can get -ve counts
    
    return X.mean(axis=0)
    
tau_leaping(reactions, initial_counts, T, n, N)

3891.78
